In [1]:
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing import image

In [2]:
import pandas as pd
train_image = []
train_y=[]
names=['cats','dogs','horses','humans']
for index, name in enumerate(names):
    for file in glob.glob(name+"\\*.jpg"):
        file_name=os.path.basename(file)
        file_name=name+'/'+file_name
        img = image.load_img(file_name,target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        train_image.append(img)
        train_y.append(index)
X = np.array(train_image)
Y = np.array(train_y)
Y = pd.Series(Y)
Y = pd.get_dummies(Y)
print(X.shape,Y.shape)

(808, 224, 224, 3) (808, 4)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.1)

In [4]:
import keras
model= keras.applications
conv_model = model.VGG19(weights='imagenet',  include_top=False, input_shape=(224,224,3))

In [5]:
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dense(16, activation='relu')(x)
x = keras.layers.Dense(8, activation='relu')(x)
predictions = keras.layers.Dense(4, activation='sigmoid')(x)
final_model_1 = keras.models.Model(inputs=conv_model.input, outputs=predictions)
final_model_1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [6]:
for layer in conv_model.layers:
    layer.trainable = False

In [7]:
conv_model.trainable = False

final_model_1.compile(optimizer="adam",  # Very low learning rate
              loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
final_model_1.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
23/23 [==============================] - 167s 7s/step - loss: 0.9599 - accuracy: 0.5997 - val_loss: 0.6621 - val_accuracy: 0.6790
Epoch 2/10
23/23 [==============================] - 178s 8s/step - loss: 0.4088 - accuracy: 0.8459 - val_loss: 0.4780 - val_accuracy: 0.7407
Epoch 3/10
23/23 [==============================] - 159s 7s/step - loss: 0.2237 - accuracy: 0.9354 - val_loss: 0.3425 - val_accuracy: 0.9012
Epoch 4/10
23/23 [==============================] - 159s 7s/step - loss: 0.1029 - accuracy: 0.9876 - val_loss: 0.2454 - val_accuracy: 0.9012
Epoch 5/10
23/23 [==============================] - 157s 7s/step - loss: 0.0568 - accuracy: 0.9931 - val_loss: 0.2195 - val_accuracy: 0.8889
Epoch 6/10
23/23 [==============================] - 159s 7s/step - loss: 0.0231 - accuracy: 1.0000 - val_loss: 0.2171 - val_accuracy: 0.8889
Epoch 7/10
23/23 [==============================] - 157s 7s/step - loss: 0.0145 - accuracy: 1.0000 - val_loss: 0.1343 - val_accuracy: 0.9753
Epoch 8/10
23

In [9]:
final_model_1.save("model.h5")

In [10]:
from keras.models import load_model
import tensorflow as tf

In [11]:
mod = load_model("model.h5")

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(mod)

In [13]:
tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\Nikita\AppData\Local\Temp\tmpoo8h7pbq\assets


In [14]:
with open('model.tflite','wb') as f:
    f.write(tflite_model)